In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from keras.models import Model
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 1. Initialize functions and generator for training and evaluation

In [2]:
def myGenerator(gen1, gen2):
    """Combine generators of imbalanced datasets into one.
    
    Args:
        gen1 (generator): First generator of image data split
            by classes
        gen2 (generator): Second generator of image data split
            by classes
        
    Returns:
        gen3 (generator): Generator that combines 2 generators
    """
    while True:
        img_data1,label1 = next(gen1)
        img_data2,label2 = next(gen2)
        if ((label1.shape[0] + label2.shape[0]) % 32) != 0:
            img_data1, label1 = next(gen1)
            img_data2, label2 = next(gen2)
        img_data_c = np.concatenate((img_data1, img_data2))

        label_size = label1.shape[1] + label2.shape[1]
        new_labels1 = np.zeros((label1.shape[0], label_size))
        new_labels1[:, :-label2.shape[1]] = label1
        new_labels2 = np.zeros((label2.shape[0], label_size))
        new_labels2[:, label1.shape[1]:] = label2

        labels_c = np.concatenate((new_labels1, new_labels2), axis=0)

        shuffler = np.random.permutation(labels_c.shape[0])
        labels_c_shuffled = labels_c[shuffler]
        img_data_c_shuffled = img_data_c[shuffler]

        yield img_data_c_shuffled, labels_c_shuffled

def plot_accuracy(history_obj, title):
    """Plots trained ML model accuracy scores.
    
    Args:
        history_df (obj): Output object from trained ML model 
        title (str): The desired title of the plot
        
    Returns:
        None
    """
    data_acc_df = pd.DataFrame({
        'epoch':[*range(1, len(
            history_obj.history['categorical_accuracy']) + 1)],
        'accuracy':history_obj.history['categorical_accuracy']}) 
    data_acc_df['Data'] = 'training'
    data_vacc_df = pd.DataFrame({
        'epoch':[*range(1, len(
            history_obj.history['val_categorical_accuracy']) + 1)],
        'accuracy':history_obj.history['val_categorical_accuracy']})
    data_vacc_df['Data'] = 'validation'
    data_m_df = pd.concat((data_acc_df, data_vacc_df), axis=0)

    plt.figure(figsize=(12, 5))
    sns.lineplot(data=data_m_df, x='epoch', y='accuracy', hue='Data')
    plt.xlabel('Epoch', fontsize=16)
    plt.ylabel('Accuracy', fontsize=16)
    plt.title(title, fontsize=18)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.ylim((0.1, 0.9))
    plt.show()
    
# Set up plot styling
plt.style.use('fivethirtyeight')
plt.style.use('seaborn-notebook')

In [3]:
# Initialize training generator
train_datagen= ImageDataGenerator(
        rotation_range=90,
        horizontal_flip=True,
        vertical_flip=True,
        rescale=1./255,
        fill_mode='nearest',
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=30,
        zoom_range=0.1,
        brightness_range=[0.7, 1.0])
train_generator_1 = train_datagen.flow_from_directory(
        'model_dataset/train/augment',
        target_size=(200, 200),
        batch_size=26,
        class_mode='categorical',
        shuffle=True,
        seed=15)
train_generator_2 = train_datagen.flow_from_directory(
        'model_dataset/train/no_augment',
        target_size=(200, 200),
        batch_size=6,
        class_mode='categorical',
        shuffle=True,
        seed=15)
train_generator = myGenerator(train_generator_1, train_generator_2)

# Initialize validation generator
valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory(
        'model_dataset/validation',
        target_size=(200, 200),
        batch_size=32,
        shuffle = True,
        class_mode='categorical')

# Initialize test generator
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    directory='model_dataset/test',
    target_size=(200, 200),
    batch_size=1,
    class_mode='categorical',
    shuffle=False,
    seed=42)

Found 280 images belonging to 4 classes.
Found 440 images belonging to 1 classes.
Found 75 images belonging to 5 classes.
Found 75 images belonging to 5 classes.


In [4]:
steps_per_epoch = 10

# 2. Train CNN model for feature extraction

In [5]:
# Weight Layers: 7 | Activation: Relu | Max Filter: 256 | Max FCN: 128
feat_extract = models.Sequential()
feat_extract.add(layers.Conv2D(64, (3, 3), activation='relu', 
                        input_shape=(200, 200, 3)))
feat_extract.add(layers.Conv2D(64, (3, 3), activation='relu'))
feat_extract.add(layers.MaxPooling2D((2, 2)))
feat_extract.add(layers.Conv2D(128, (3, 3), activation='relu'))
feat_extract.add(layers.Conv2D(128, (3, 3), activation='relu'))
feat_extract.add(layers.MaxPooling2D((2, 2)))
feat_extract.add(layers.Conv2D(256, (3, 3), activation='relu'))
feat_extract.add(layers.Conv2D(256, (3, 3), activation='relu'))
feat_extract.add(layers.MaxPooling2D((2, 2)))
feat_extract.add(layers.Flatten())

x = feat_extract.output
x = layers.Dense(128, activation='relu')(x)
x =layers.Dropout(0.2)(x)
prediction_layer = layers.Dense(5, activation = 'softmax')(x)

# Train CNN model
vgg16 = Model(inputs=feat_extract.input, outputs=prediction_layer)
vgg16.compile(loss='categorical_crossentropy', optimizer='rmsprop',
              metrics=['CategoricalAccuracy'])

history_vgg16 = vgg16.fit(train_generator, steps_per_epoch=steps_per_epoch,
                          epochs=75, validation_data=valid_generator,
                          validation_steps=2, verbose=0)

In [6]:
# Extract train features for SVM model
for i in range(steps_per_epoch):
    (X_sub, y_sub) = next(train_generator)
    y_sub = np.argmax(y_sub, axis=1)
    if i == 0:
        X_train = X_sub.copy()
        y_train = y_sub.copy()
    else:
        X_train = np.append(X_train, X_sub, axis=0)
        y_train = np.append(y_train, y_sub, axis=0)
X_ext_train = feat_extract.predict(X_train)

# Extract validation features for SVM model
for i in range(3):
    (X_sub, y_sub) = next(valid_generator)
    y_sub = np.argmax(y_sub, axis=1)
    if i == 0:
        X_val = X_sub.copy()
        y_val = y_sub.copy()
    else:
        X_val = np.append(X_val, X_sub, axis=0)
        y_val = np.append(y_val, y_sub, axis=0)
X_ext_val = feat_extract.predict(X_val)

# Extract test features for SVM model
y_test = test_generator.classes
for i in range(test_generator.samples):
    (X_sub, _) = next(test_generator)
    if i == 0:
        X_test = X_sub.copy()
    else:
        X_test = np.append(X_test, X_sub, axis=0)
X_ext_test = feat_extract.predict(X_test)

# 3. Evaluate SVM model

In [7]:
c_labels = list(test_generator.class_indices.keys())

## Without grid search

In [8]:
# Initialize SVM classifier
svm = SVC(class_weight='balanced')
svm.fit(X_ext_train, y_train)

# Check for overfitting
y_train_pred = svm.predict(X_ext_train)
print(classification_report(y_train, y_train_pred,
                               target_names=c_labels))

               precision    recall  f1-score   support

  dried_basil       0.48      0.47      0.48        64
dried_oregano       0.39      0.58      0.47        69
dried_parsley       0.84      0.52      0.64        62
  dried_thyme       0.39      0.34      0.36        65
       random       0.89      0.90      0.89        60

     accuracy                           0.56       320
    macro avg       0.60      0.56      0.57       320
 weighted avg       0.59      0.56      0.56       320



In [9]:
# Evaluate model
y_val_pred = svm.predict(X_ext_val)
print(classification_report(y_val, y_val_pred,
                               target_names=c_labels))

               precision    recall  f1-score   support

  dried_basil       0.00      0.00      0.00        15
dried_oregano       0.28      0.73      0.41        15
dried_parsley       0.83      0.33      0.48        15
  dried_thyme       0.38      0.20      0.26        15
       random       0.80      0.80      0.80        15

     accuracy                           0.41        75
    macro avg       0.46      0.41      0.39        75
 weighted avg       0.46      0.41      0.39        75



## With grid search

In [10]:
# Initialize SVM classifier
svm = SVC(class_weight='balanced')
grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001, 'scale', 'auto'],
              'kernel': ['linear', 'poly', 'rbf']}
grid_search = GridSearchCV(estimator=svm, param_grid=grid,
                         cv=5, n_jobs=-1, verbose=2)

# Fit grid search
grid_search.fit(X_ext_train, y_train)
print('Best Parameters:')
print(grid_search.best_params_)

# Check for overfitting
y_train_pred = svm.predict(X_ext_train)
print(classification_report(y_train, y_train_pred,
                               target_names=c_labels))

Fitting 5 folds for each of 105 candidates, totalling 525 fits


KeyboardInterrupt: 

In [ ]:
# Evaluate model
y_val_pred = svm.predict(X_ext_val)
print(classification_report(y_val, y_val_pred,
                               target_names=c_labels))

In [ ]:
# {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}

# 4. Evaluate best SVM model

In [ ]:
# Initialize SVM classifier
svm = SVC(class_weight='balanced', C=1, gamma=0.1, kernel='rbf')
svm.fit(X_ext_train, y_train)

# Check for overfitting
y_train_pred = svm.predict(X_ext_train)
print(classification_report(y_train, y_train_pred,
                               target_names=c_labels))

In [ ]:
# Evaluate model
y_test_pred = svm.predict(X_ext_test)
print(classification_report(y_test, y_test_pred,
                               target_names=c_labels))

# Display confusion matrix
cm = confusion_matrix(y_test, y_test_pred)
df_cm = pd.DataFrame(cm, index=c_labels, columns=c_labels)
plt.figure(figsize=(10, 7))
sns.heatmap(df_cm, annot=True, cmap=plt.cm.Blues)
plt.ylabel('True', fontsize=14)
plt.xlabel('Predicted', fontsize=14)
plt.show()